In [1]:
# Write /cmd_vel drift into robot_moves.py using your measured values
import re, importlib, datetime, shutil
import robot_moves as rm
importlib.reload(rm)

rm_path = rm.__file__

# Your proven values:
RATE_HZ  = 20.0
GAP      = 0.35
VY_LEFT  = 0.200
WZ_LEFT  = 0.650
VY_RIGHT = -0.200
WZ_RIGHT = -0.845

block = f"""
# === DRIFT (auto-generated CMD_VEL) BEGIN ===
# Drift via /cmd_vel (vy + wz), using values measured on this robot.
DRIFT_RATE_HZ:  float = {RATE_HZ}
DRIFT_GAP:      float = {GAP}
DRIFT_VY_LEFT:  float = {VY_LEFT}
DRIFT_WZ_LEFT:  float = {WZ_LEFT}
DRIFT_VY_RIGHT: float = {VY_RIGHT}
DRIFT_WZ_RIGHT: float = {WZ_RIGHT}

import rclpy as _rm_rclpy
from geometry_msgs.msg import Twist as _rm_Twist
from rclpy.node import Node as _rm_Node

__rm_drift_node = None
__rm_cmdvel_pub = None

def _rm_ensure_cmdvel():
    global __rm_drift_node, __rm_cmdvel_pub
    if __rm_cmdvel_pub is not None:
        return
    if not _rm_rclpy.ok():
        _rm_rclpy.init(args=None)
    __rm_drift_node = _rm_Node("robot_moves_cmdvel")
    __rm_cmdvel_pub = __rm_drift_node.create_publisher(_rm_Twist, "/cmd_vel", 10)

def _rm_hold_cmdvel(seconds: float, vy: float, wz: float, rate_hz: float = DRIFT_RATE_HZ):
    _rm_ensure_cmdvel()
    import time as _t
    end = _t.time() + float(seconds)
    dt  = 1.0 / float(rate_hz)
    while _t.time() < end:
        m = _rm_Twist()
        m.linear.y  = float(vy)
        m.angular.z = float(wz)
        __rm_cmdvel_pub.publish(m)
        _t.sleep(dt)
    __rm_cmdvel_pub.publish(_rm_Twist())  # stop
    if DRIFT_GAP and DRIFT_GAP > 0:
        _t.sleep(DRIFT_GAP)

def drift_left(seconds: float = 1.2):
    \"\"\"Smooth drift left (strafe + yaw) via /cmd_vel.\"\"\"
    _rm_hold_cmdvel(seconds, vy=DRIFT_VY_LEFT,  wz=DRIFT_WZ_LEFT)

def drift_right(seconds: float = 1.2):
    \"\"\"Smooth drift right (strafe + yaw) via /cmd_vel.\"\"\"
    _rm_hold_cmdvel(seconds, vy=DRIFT_VY_RIGHT, wz=DRIFT_WZ_RIGHT)
# === DRIFT (auto-generated CMD_VEL) END ===
""".lstrip("\n")

src = open(rm_path, "r", encoding="utf-8").read()

# Replace any previous auto-generated DRIFT block (CMD_VEL or older), else append.
new_src = re.sub(r"# === DRIFT \(auto-generated.*?END ===", block, src, flags=re.S)
if new_src == src:
    new_src = src.rstrip() + "\n\n" + block

backup = rm_path.replace(".py", f".backup.{datetime.datetime.now():%Y%m%d-%H%M%S}.py")
shutil.copy2(rm_path, backup)
with open(rm_path, "w", encoding="utf-8") as f:
    f.write(new_src)

importlib.invalidate_caches()
importlib.reload(rm)
try: rm.set_rate(40)
except: pass

print(f"Patched {rm_path} with /cmd_vel drifts.")
print(f"Backup saved at {backup}. Now use: rm.drift_left(t) / rm.drift_right(t)")


Patched /home/ubuntu/Matamoe/robot_moves.py with /cmd_vel drifts.
Backup saved at /home/ubuntu/Matamoe/robot_moves.backup.20250915-093432.py. Now use: rm.drift_left(t) / rm.drift_right(t)


In [2]:
import importlib, robot_moves as rm
importlib.reload(rm)

rm.drift_left(1.2)
rm.drift_right(1.2)
rm.stop()


In [3]:
rm.forward(2)